In [ ]:
@patch("rss_collector.utils.get_proxy_settings")
@patch("rss_collector.proxy.fetch_feed_content_with_proxy")
@patch("rss_collector.feed_parser.parse_feed")
def test_process_feeds(
    mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings, test_feed_urls
):
    """
    Test the full process_feeds functionality with all components integrated.
    """
    # Mock proxy settings
    mock_get_proxy_settings.return_value = mock_proxy_settings

    # Mock fetch_feed_content_with_proxy to return mock feed content
    mock_fetch_feed.return_value = "Mock RSS Feed Content"

    # Mock parse_feed to return the mocked feed data
    mock_parse_feed.return_value = mock_feed_data

    # Call the process_feeds function
    process_feeds(test_feed_urls)

    # Assertions for get_proxy_settings
    mock_get_proxy_settings.assert_called_once()

    # Assertions for fetch_feed_content_with_proxy
    assert mock_fetch_feed.call_count == len(test_feed_urls)
    for feed_url in test_feed_urls:
        mock_fetch_feed.assert_any_call(feed_url, mock_proxy_settings["app_proxy"], mock_proxy_settings["username"], mock_proxy_settings["password"])

    # Assertions for parse_feed
    assert mock_parse_feed.call_count == len(test_feed_urls)
    for _ in test_feed_urls:
        mock_parse_feed.assert_any_call("Mock RSS Feed Content")

    # Verify JSON file creation
    output_dir = "rss_metadata"
    saved_files = os.listdir(output_dir)
    assert len(saved_files) == 1  # Only one consolidated JSON file should be created

    # Validate JSON content
    output_file = os.path.join(output_dir, saved_files[0])
    with open(output_file, "r") as f:
        metadata = json.load(f)

    assert "timestamp" in metadata
    assert "articles" in metadata
    assert len(metadata["articles"]) == len(mock_feed_data["entries"])

    # Cleanup created files
    for file in saved_files:
        os.remove(os.path.join(output_dir, file))
    os.rmdir(output_dir)
